IMPORTS

In [2]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 3.1 MB/s eta 0:00:00


In [3]:
import os
import polars as pl
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


SETTING PATH

In [4]:
# Change working directory to your specific folder
os.chdir('/content/drive/MyDrive/YouTube_Semantic_Search_ML/Data Science')

LOAD DATA

In [5]:
df = pl.read_parquet('data/video-transcripts.parquet')
df.head()

video_id,datetime,title,description,transcript
str,datetime[μs],str,str,str
"""7Oy2NmPwJXo""",2024-09-26 23:24:35,"""I Quit My Job… Here’s How Much…","""Here, I share all 9 ways I scr…","""14 months ago I made a big lif…"
"""ZVVkdXHqEuM""",2024-09-23 15:45:12,"""Knowledge Distillation Explain…","""""","""knowledge distillation explain…"
"""reXoKNC_Wx4""",2024-09-20 18:15:44,"""Quantization Explained in 60 S…","""""","""here's quantization explained …"
"""9joIFeKuf04""",2024-09-16 14:01:44,"""Python Explained in 60 Seconds…","""""","""here's python explained in 60 …"
"""pNg2DJ4spXg""",2024-09-12 16:36:12,"""Python QuickStart for People L…","""Here, I share a beginner Quick…","""python is the go-to programmin…"


EMBED TITLES AND TRANSCRIPTS

In [6]:
model_name = 'all-MiniLM-L6-v2'
column_name_list = ['title', 'transcript']

In [7]:
model = SentenceTransformer(model_name)

for column_name in column_name_list:
    # generate embeddings
    embedding_arr = model.encode(df[column_name].to_list())

    # store embeddings in a dataframe
    schema_dict = {column_name+'_embedding-'+str(i): float for i in range(embedding_arr.shape[1])}
    df_embedding = pl.DataFrame(embedding_arr, schema=schema_dict)

    # append embeddings to video index
    df = pl.concat([df, df_embedding], how='horizontal')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
df_embedding.shape

(106, 384)

In [9]:
df.shape

(106, 773)

In [10]:
df.head()

video_id,datetime,title,description,transcript,title_embedding-0,title_embedding-1,title_embedding-2,title_embedding-3,title_embedding-4,title_embedding-5,title_embedding-6,title_embedding-7,title_embedding-8,title_embedding-9,title_embedding-10,title_embedding-11,title_embedding-12,title_embedding-13,title_embedding-14,title_embedding-15,title_embedding-16,title_embedding-17,title_embedding-18,title_embedding-19,title_embedding-20,title_embedding-21,title_embedding-22,title_embedding-23,title_embedding-24,title_embedding-25,title_embedding-26,title_embedding-27,title_embedding-28,title_embedding-29,title_embedding-30,title_embedding-31,…,transcript_embedding-347,transcript_embedding-348,transcript_embedding-349,transcript_embedding-350,transcript_embedding-351,transcript_embedding-352,transcript_embedding-353,transcript_embedding-354,transcript_embedding-355,transcript_embedding-356,transcript_embedding-357,transcript_embedding-358,transcript_embedding-359,transcript_embedding-360,transcript_embedding-361,transcript_embedding-362,transcript_embedding-363,transcript_embedding-364,transcript_embedding-365,transcript_embedding-366,transcript_embedding-367,transcript_embedding-368,transcript_embedding-369,transcript_embedding-370,transcript_embedding-371,transcript_embedding-372,transcript_embedding-373,transcript_embedding-374,transcript_embedding-375,transcript_embedding-376,transcript_embedding-377,transcript_embedding-378,transcript_embedding-379,transcript_embedding-380,transcript_embedding-381,transcript_embedding-382,transcript_embedding-383
str,datetime[μs],str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""7Oy2NmPwJXo""",2024-09-26 23:24:35,"""I Quit My Job… Here’s How Much…","""Here, I share all 9 ways I scr…","""14 months ago I made a big lif…",-0.032602,0.05473,0.062131,0.021577,0.032131,-0.029671,0.060126,0.017054,-0.020178,-0.041739,-0.028116,-0.043028,-0.009215,-0.008168,-0.114184,0.036235,-0.019726,-0.007087,-0.043536,-0.123903,0.018751,0.043346,-0.036682,0.053513,0.085077,0.079149,0.002353,0.059304,-0.010035,-0.002053,0.011399,0.078917,…,0.052521,-0.035608,-0.01663,-0.017407,-0.089023,-0.035869,-0.022163,0.021232,-0.15436,0.056134,0.005767,-0.015907,0.042088,0.037927,0.013388,0.032578,0.043277,-0.056996,-0.020893,-0.048742,-0.020756,-0.002496,0.014135,-0.013788,-0.026399,0.024453,0.014255,0.061749,0.018591,-0.038786,0.055462,-0.117372,0.083418,0.067269,-0.09923,-0.062003,-0.022241
"""ZVVkdXHqEuM""",2024-09-23 15:45:12,"""Knowledge Distillation Explain…","""""","""knowledge distillation explain…",-0.011732,-0.04271,0.088132,0.034729,-0.012911,-0.05419,0.012225,-0.049659,-0.000252,-0.049076,-0.036361,0.02852,-0.012546,-0.002948,-0.102184,0.018863,-0.028445,0.060415,-0.18645,-0.11647,0.017884,-0.026459,-0.048636,0.00278,0.058819,0.033135,0.001362,-0.005529,0.081662,-0.060976,0.055198,0.03866,…,0.183374,-0.010083,0.001357,-0.065074,-0.022382,-0.044083,-0.017056,0.027268,0.079017,-0.040486,-0.09567,-0.023765,0.000538,-0.055429,-0.011123,-0.087084,0.04548,0.001493,0.052873,-0.046608,-0.024501,0.003573,0.147117,0.055381,0.02479,0.008054,-0.078529,0.037899,0.025007,-0.043297,-0.036289,-0.054006,-0.068295,0.07171,0.048973,0.080141,-0.012561
"""reXoKNC_Wx4""",2024-09-20 18:15:44,"""Quantization Explained in 60 S…","""""","""here's quantization explained …",-0.017366,0.003107,0.042879,-0.018499,-0.015287,0.04136,0.042744,-0.040049,0.072668,-0.031978,0.045571,-0.070918,-0.060834,-0.000611,-0.055284,0.00995,0.002527,0.039422,-0.155163,-0.07763,0.075676,-0.047264,-0.056142,-0.031138,0.084492,0.086777,0.002662,-0.04727,0.118983,-0.006494,0.003519,0.031334,…,0.048561,0.005305,0.007814,-0.03517,0.057467,0.050811,0.080404,0.018837,0.031748,-0.094081,-0.069966,-0.038736,-0.113055,-0.057861,-0.0

SAVE INDEX TO FILE

In [11]:
df.write_parquet('data/video-index.parquet')